# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Согласно исходных данных, файл с данным, предоставленный для анализа имеет следующий путь к файлу: /datasets/data.csv. Откроем и рассмотрим предоставленные данные.

In [1]:
import pandas as pd
import warnings


warnings.filterwarnings("ignore")

polluted_data = pd.read_csv('/datasets/data.csv')
#Просмотрим общую информацию о данных
polluted_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
#Просмотрим данные функцией describle
polluted_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [3]:
#Проверим первые строки информации для проверки категориальных переменных
polluted_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Данные в целом достаточны для исследования, но очевидно имеют пропуски например в столбцах - total_income и days_employed (которые возможно ещё и совпадают), скорее всего имеют дубликаты. Кроме того максимальное число детей в 20, отрицательные количества детей и стажа, а также стаж в 1100 лет выглядят странно. Кроме того очевидно, что в стобце education есть дубликаты (например Среднее и среднее), также столбец total_income не выглядит удобным для анализа, его лучше сделать int

## Шаг 2. Предобработка данных

В начале работы продумаем и проверим все столбцы в не зависимости от того, нужны нам они или нет.

### Обработка пропусков

**Столбец children - количество детей в семье.** Проверим какие уникальные значения у нас есть и сколько их.

In [4]:
#проверяем уникальные значения и заодно отслеживаем пропуски
print(polluted_data['children'].value_counts(dropna=False))
#посмотрим строки с значением детей 20
polluted_data.loc[polluted_data['children'] == 20].head(20)

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
3302,20,NaN,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,NaN,профильное образование
3396,20,NaN,56,высшее,0,женат / замужем,0,F,компаньон,0,NaN,высшее образование
3671,20,-913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
3697,20,-2907.910616,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.694664,на покупку подержанного автомобиля
3735,20,-805.044438,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью


Для нас этот столбец важен. Мы не можем просто отбросить неправильные значения.
Это в целом категориальная величина – у неё есть разумные значения. Проверяем его значения – из странного есть -1 и 20. -1 скорее всего полностью случайная опечатка. Я бы прибавил это число к 1 ребенку. 
20 проверяю на зависимости, вроде ни от чего не зависит. Нам важно наличие детей, возможно это также полностью случайная опечатка – пусть это будет 2
 
Исправим данные значения:

In [5]:
polluted_data.loc[polluted_data['children'] == -1,'children'] = 1  # вместо -1 ставим 1
polluted_data.loc[polluted_data['children'] == 20,'children'] = 2  # вместо 20 ставим 2
print(polluted_data['children'].value_counts(dropna=False)) # проверяем результат

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


**Столбец days_employed** - трудовой стаж в днях. 
Это количественный столбец, посмотрим, что тут есть на примере первых 10 строк и проверим количество отсутсвующих значений

In [6]:
# подсчитаем количество пропусков
print('Пропусков в таблице: ',len(polluted_data[polluted_data['days_employed'].isna()]))
# коррелируют ли пропуски с другими столбцами
print('Корреляция:')
print(polluted_data[polluted_data['total_income'].isna()].count())
#проверяем данные столбца days_employed на примере первых 10 строк
print('Типичные данные:')
polluted_data.head(10)

Пропусков в таблице:  2174
Корреляция:
children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64
Типичные данные:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Налицо наличие отсутствующих значений, корреляция с столбцом total_income, странные отрицательные стажи и очень большой разброс в размерах стажа.

Для решения поставленных вопросов нам не требуется данный столбец и я не буду его использовать

При этом если бы стояла задача по его обработке то:

-- Отсутствующие значения стажа и отрицательное количество стажа видимо полностью случайные ошибки. Корреляция между отсутсвием стажа и total_income странна, скорее всего погрешность возникла сразу в двух столбцах. Отрицательным значениям столбца мы меняем знак.Отсутсвующие заменяем медианой. При этом если сразу заменить значение на медиану, то из-за аномальных цифр будет плохо.       

-- Интересно рассмотреть большие числа в стаже - если предположить, что человек не может работать ранее 18 лет со стажем, и на пенсию уходит обучно в 60 лет, то максимальное число рабочих дней около 15330 (42*365). Всё что больше – артефакт.
Но возможно эти данные даны нам не в днях? Возможно это часы и тогда эти значения в максимуме будут близки к 367920 (15330*24) и данные числа будет в тех случаях, когда клиенты пенсионеры. Проверим данную версию: 

In [7]:
# посмотрим, сколько данных выглядит странно
strange_days=polluted_data[polluted_data['days_employed'] > 15300]
print(strange_days['income_type'].head(15))

4     пенсионер
18    пенсионер
24    пенсионер
25    пенсионер
30    пенсионер
35    пенсионер
50    пенсионер
56    пенсионер
71    пенсионер
78    пенсионер
86    пенсионер
87    пенсионер
88    пенсионер
98    пенсионер
99    пенсионер
Name: income_type, dtype: object


Версия подтверждается, это **стаж в часах**, и это неслучайные ошибки. Если бы нам были важны эти данные то необходимо было бы привести стаж пенсионеров к рабочим дням после чего определить медианные значения и присвоить их для отсутствующих. Но поскольку стаж нам не важен,  то я далее не хочу использовать данный столбец и буду работать с данными без этого столбца.

In [8]:
# выделим сокращенную таблицу, которую мы будем использовать в дальнейшем
data = polluted_data[['children','dob_years','education','education_id','family_status','family_status_id','gender','income_type','debt','total_income','purpose']]
# проверим столбцы
print(data.info())
# проверим первые строки 
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB
None


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Столбец dob_years** - возраст клиента в годах. Это я думаю категориальный столбец, значения должны лежать между 18 и 100 годами и быть целыми числами. Проверим какие уникальные значения у нас есть, а также есть ли у нас пропуски

In [9]:
print(data['dob_years'].unique())  #проверяем уникальные значения
print(data[data['dob_years'].isnull()].head()) # отслеживаем пропуски

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
Empty DataFrame
Columns: [children, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


Пропусков нет, но есть возраст = 0 который точно неправильный. Данный столбец нам неважен, но его стоит сохранить для возможной проверки столбца о типе занятости. Выясним, сколько значений 0 у нас есть. Если немного, то высчитаем медианное значение и заменим 0 на медиану.

In [10]:
# посчитаем количество заёмщиков с неправильным возрастом
print('Количество заёмщиков с возрастом 0: ',data.loc[data['dob_years'] == 0,'dob_years'].count())

# посчитаем медианный возраст, сразу переведем его в целые числа
mean_dob_years = data['dob_years'].median()
mean_dob_years = mean_dob_years.round(0).astype(int)

print ('Средний возраст заёмщика: ',mean_dob_years)

Количество заёмщиков с возрастом 0:  101
Средний возраст заёмщика:  42


In [11]:
#проверим какой у заёмщиков с возрастом 0 тип дохода и образование
newborn_borrower = data.query('dob_years == 0')
print('Тип дохода заёмщиков с возрастом 0: ')
print(newborn_borrower['income_type'].value_counts())

print('Образование заёмщиков с возрастом 0: ')
print(newborn_borrower['education_id'].value_counts())
#выведем и посмотрим первые строки 
newborn_borrower.head()

Тип дохода заёмщиков с возрастом 0: 
сотрудник      55
компаньон      20
пенсионер      20
госслужащий     6
Name: income_type, dtype: int64
Образование заёмщиков с возрастом 0: 
1    64
0    35
2     2
Name: education_id, dtype: int64


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль


In [12]:
# поскольку случаев новорожденных заёмщиков невелико и большая часть из них не пенсионеры,
# то заменим 0 на медиану
data.loc[data['dob_years'] == 0,'dob_years'] = mean_dob_years
# проверим возраст
print('Количество заёмщиков с возрастом 0: ',data.loc[data['dob_years'] == 0,'dob_years'].count())

Количество заёмщиков с возрастом 0:  0


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


**Столбец education и Столбец education_id**  (образование клиента и идентификатор образования) я буду рассматривать вместе. Это категориальные столбцы связанные друг с другом. Проверим какие уникальные значения у нас есть, а также есть ли у нас пропуски. После можно будет сверить эти столбцы по количеству. Эти столбцы могут быть для нас важны.

In [13]:
# проверим education, поищем пропуски
print(data['education'].value_counts(dropna=False))
# проверим education_id, поищем пропуски
print(data['education_id'].value_counts(dropna=False))

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64
1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


Пропусков нет. Но очевидно, что в столбце education множество дубликатов. В education_id можно определить закладываемые значения 
0 – высшее образование
1 - среднее образование
2 - неоконченное высшее образование
3 – начальное образование
4 – ученая степень
Проверить совпадение количества этих столбцов можно будет только после решения вопроса с дубликатами.

**Столбец family_status и Столбец family_status_id** (семейное положение и идентификатор семейного положения) я буду рассматривать вместе. Это категориальные столбцы связанные друг с другом. Проверим какие уникальные значения у нас есть, а также есть ли у нас пропуски. После можно будет сверить эти столбцы по количеству. Эти столбцы могут быть для нас важны.

In [14]:
# проверим family_status, поищем пропуски
print(data['family_status'].value_counts(dropna=False))
# проверим family_status_id, поищем пропуски
print(data['family_status_id'].value_counts(dropna=False))

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


Пропусков нет. Дубликатов в столбце family_status нет, но стоит привести запись к одному виду со строчными буквами. В family_status_id можно определить закладываемые значения 
0 – женат/замужем
1 - гражданский брак
2 - вдова/вдовец
3 – в разводе
4 – не женат/не замужем
Мы можем проверить совпадение этих столбцов по количеству.
Всё совпадает и это редкость :-)

**Стобец gender** — пол клиента
Это категориальный столбец. Проверим какие уникальные значения у нас есть, а также есть ли у нас пропуски. Этот столбец не очень важен для нас.

In [15]:
# проверим gender, поищем пропуски
print(data['gender'].value_counts(dropna=False))

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Пропусков нет. Дубликатов в столбце gender нет, но есть странное значение "XNA". Поскольку для наших вопросов пол не имеет важного значения, то можно оставить данное значение и так, либо удалить данную строку но возможно интересно будет посмотреть есть ли зависимость между полом и задаваемыми нам вопросами. Поскольку заёмщиков женщин примерно в два раза больше, то вероятность того, что XNA женщина в два раза больше. Перед переименованием посмотрим данную строку поподробнее:

In [16]:
# проверим строку содержащую gender XNA
data.loc[data['gender'] == 'XNA']

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Строка за исключением пола выглядит хорошо. :-) хороший общий доход, интересная тема займа, нет просрочек с платежами. Стоит оставить эту строку. Меняю пол на F, поскольку для целей данного исследования возможно стоит оценить пол с точки зрения ответов на вопросы

In [17]:
# меняем строку содержащую gender XNA
data.loc[data['gender'] == 'XNA','gender'] = 'F'  # вместо XNA ставим F
print(data['gender'].value_counts(dropna=False)) # проверяем результат

F    14237
M     7288
Name: gender, dtype: int64


**Столбец income_type** - тип занятости. Столбец категориальный и может быть для нас важен

In [18]:
# проверим income_type, поищем пропуски
print(data['income_type'].value_counts(dropna=False))

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64


Пропусков нет. Непонятно что такое "компаньон" но в целом для нас это неважно. 

**Столбец debt** - имел ли клиент задолженность по возврату кредитов, пожалуй самый важный для нас. Это критериальный столбец. Проверим нет ли здесь пропусков и странных значений

In [19]:
print(data['debt'].value_counts(dropna=False))

0    19784
1     1741
Name: debt, dtype: int64


Пропусков нет, 0 - задолженностей нет, 1 - задолженности были. Столбец пригоден для анализа

**Столбец total_income** -доход в месяц. Это количественный столбец, посмотрим, что тут есть на примере первых и последних строк, проверим наличие отрицательных значений и проверим количество отсутсвующих значений. Столбец важен нам.

In [20]:
# проверим total_income на обычные данные
print('Обычные значения выглядят как: ')
print(data['total_income'].head())
print(data['total_income'].tail())
# проверим отрицательные значения и поищем пропуски
strange_tot=data[data['total_income'] <=0]
print('Отрицательных значений:',strange_tot['total_income'].count())
print('Пропусков в таблице: ',len(data[data['total_income'].isna()]))

Обычные значения выглядят как: 
0    253875.639453
1    112080.014102
2    145885.952297
3    267628.550329
4    158616.077870
Name: total_income, dtype: float64
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, dtype: float64
Отрицательных значений: 0
Пропусков в таблице:  2174


Пропусков 2174, это много для того, чтобы их вычеркнуть. Думаю заменить их на медианные значения исходя из типа дохода. Затем думаю изменить класс данных на целые - с ними нагляднее работать. 

In [21]:
#получим средние значения исходя из типа занятости
median_income_type = data[data['total_income']!='NaN'].pivot_table(
    index='income_type', values='total_income')
median_dict = median_income_type.reset_index()
median_dict.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,income_type,total_income
0,безработный,131339.751676
1,в декрете,53829.130729
2,госслужащий,170898.309923
3,компаньон,202417.461462
4,пенсионер,137127.465690


In [22]:
#соеденим средние значения с основной таблицей
data = data.merge(median_dict, on =['income_type'],how = 'left')
data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_x,purpose,total_income_y
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,161380.260488
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,161380.260488
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,161380.260488
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,161380.260488
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,137127.465690


In [23]:
#сделаем функцию по подставлению измененных данных
def fin_inc(row):
    total_income = row['total_income_x']
    median_incomes = row['total_income_y']
    if total_income != 0:
        return total_income
    else:
        return median_incomes

#почему-то это не работает с NaN, я делаю промежуточное преобразование NaN на 0
data['total_income_x']  = data['total_income_x'].fillna(value=0)

# делаю сведение значений
data['total_income'] = data.apply(fin_inc, axis =1)
print('Пропусков в таблице: ',len(data[data['total_income'].isna()]))

# меняем тип данных
data['total_income']=data['total_income'].astype(int)

# проверяем
print(data.info())
data.head()

Пропусков в таблице:  0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income_x      21525 non-null float64
purpose             21525 non-null object
total_income_y      21525 non-null float64
total_income        21525 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 2.3+ MB
None


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_x,purpose,total_income_y,total_income
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,161380.260488,253875
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,161380.260488,112080
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,161380.260488,145885
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,161380.260488,267628
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,137127.465690,158616


Пропущенных данных нет и с данными стало легче работать

**Столбец purpose** -цель получения кредита  
Данный столбец важен в наших исследованиях, но очень плохо структурирован. Стоит проверить его на пропуски и посмотреть, как он вообще выглядит.

In [24]:
# проверим purpose, поищем пропуски
print('Пропусков в таблице: ',len(data[data['purpose'].isna()]))
# проверим как вообще выглядят данные
print('Обычные значения выглядят как: ')
print(data['purpose'].head(10))
data['purpose'].tail(10)

Пропусков в таблице:  0
Обычные значения выглядят как: 
0                 покупка жилья
1       приобретение автомобиля
2                 покупка жилья
3    дополнительное образование
4               сыграть свадьбу
5                 покупка жилья
6             операции с жильем
7                   образование
8         на проведение свадьбы
9       покупка жилья для семьи
Name: purpose, dtype: object


21515                заняться образованием
21516                 покупка своего жилья
21517         на покупку своего автомобиля
21518                 сделка с автомобилем
21519    покупка коммерческой недвижимости
21520                    операции с жильем
21521                 сделка с автомобилем
21522                         недвижимость
21523         на покупку своего автомобиля
21524                на покупку автомобиля
Name: purpose, dtype: object

Пропусков нет. Но стоит привести данные к нескольким категориям.

**Вывод**

Посмотрим, как теперь выглядят данные:

In [25]:
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income_x      21525 non-null float64
purpose             21525 non-null object
total_income_y      21525 non-null float64
total_income        21525 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 2.3+ MB


,children,dob_years,education_id,family_status_id,debt,total_income_x,total_income_y,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04,21525.000000,2.152500e+04
mean,0.479721,43.490453,0.817236,0.972544,0.080883,1.505128e+05,167395.915741,1.673954e+05
std,0.755528,12.218595,0.548138,1.420324,0.272661,1.098972e+05,22013.368093,9.790696e+04
min,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000e+00,53829.130729,2.066700e+04
25%,0.000000,34.000000,1.000000,0.000000,0.000000,8.861283e+04,161380.260488,1.077980e+05
50%,0.000000,42.000000,1.000000,0.000000,0.000000,1.355147e+05,161380.260488,1.519310e+05
75%,1.000000,53.000000,1.000000,1.000000,0.000000,1.955436e+05,170898.309923,2.024170e+05
max,5.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06,499163.144947,2.265604e+06


**Вывод**

Убраны пропуски и странные значения. Есть готовность к обработке дубликатов

### Обработка дубликатов

После рассмотрения столбцов, был обнаружен столбец - education , который явно имеет много дубликатов и нуждается в чистке.

In [26]:
#посмотрим уникальные значения
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

В таком виде этот столбец оставлять нельзя. Нужно привести его к одному виду и перепроверить.

In [27]:
#методом str.lower() напишем всё строчными буквами
data['education'] = data['education'].str.lower()
print(data.groupby('education')['education'].count())
print('')
#сверимся с нашими идентефикаторами
print('идентификаторы:')
data['education_id'].value_counts()

education
высшее                  5260
начальное                282
неоконченное высшее      744
среднее                15233
ученая степень             6
Name: education, dtype: int64

идентификаторы:


1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

Похоже, данные теперь правильны.
Теперь мы можем применить к данным методы  duplicated() и drop_duplicates()

In [28]:
#проверим наличие дубликатов
print ('Дубликатов в таблице:', data.duplicated().sum())
#дубликаты могли появиться из за ошибок в столбце с описанием образования
#удалим их и восстановим индексацию
data = data.drop_duplicates().reset_index(drop=True) 
#проверим результаты
print ('Дубликатов в таблице после обработки:', data.duplicated().sum())

Дубликатов в таблице: 72
Дубликатов в таблице после обработки: 0


**Вывод**

Был проверен и сведен к единообразию столбец - education. После этого были обнаружены полные дубликаты и удалены. Затем был воостановлен порядок индексирования.

### Лемматизация

У нас есть столбец purpose который заполнялся явно без каких-то правил и который важен нам для ответа на поставленные вопросы. Мне необходимо посмотреть пропуски, найти какие-то закономерности и привести столбец в порядок:

In [29]:
#проверим какие цели у нас есть и как часто они встречаются
data['purpose'].value_counts(dropna=False)

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Очевидно что берут на: 
свадьбу, жильё, покупку автомобилей и образование.
Нам необходимо провести лемматизацию для более точного определения ключевых целей.

In [30]:
##Загружаем библиотеку и лемматизируем столбец 'purpose'
from pymystem3 import Mystem
m = Mystem()

#сделаем функцию для лемматизации столбца
def purpose_lemm(purpose_messy):
    lemma = ' '.join(m.lemmatize(purpose_messy))
    return lemma

#сделаем функцию, которая сможет сделать столбец с четкими целями кредита
def purpose_cleaning(purpose_lm):
    if 'автомобиль' in purpose_lm:
        return 'автомобиль'
    if 'свадьба' in purpose_lm:
        return  'свадьба'
    if 'образование' in purpose_lm:
        return 'образование'
    return 'недвижимость'

# сделаем промежуточный столбец для лемматизированного текста
data['purpose_lemmatize'] = data['purpose'].apply(purpose_lemm)

# сделаем столбец, который будет содержать чистые цели
data['purpose_clear'] = data['purpose_lemmatize'].apply(purpose_cleaning)     

# проверим столбцы и заполнение значений
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 15 columns):
children             21453 non-null int64
dob_years            21453 non-null int64
education            21453 non-null object
education_id         21453 non-null int64
family_status        21453 non-null object
family_status_id     21453 non-null int64
gender               21453 non-null object
income_type          21453 non-null object
debt                 21453 non-null int64
total_income_x       21453 non-null float64
purpose              21453 non-null object
total_income_y       21453 non-null float64
total_income         21453 non-null int64
purpose_lemmatize    21453 non-null object
purpose_clear        21453 non-null object
dtypes: float64(2), int64(6), object(7)
memory usage: 2.5+ MB
None


,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_x,purpose,total_income_y,total_income,purpose_lemmatize,purpose_clear
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,161380.260488,253875,покупка жилье \n,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,161380.260488,112080,приобретение автомобиль \n,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,161380.260488,145885,покупка жилье \n,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,161380.260488,267628,дополнительный образование \n,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,137127.465690,158616,сыграть свадьба \n,свадьба


In [31]:
# выделим сокращенную таблицу, которую мы будем использовать в дальнейшем
clear_data = data[['children','dob_years','education_id','family_status_id','gender','income_type','debt','total_income','purpose_clear']]
# проверим столбцы
print(clear_data.info())
# проверим первые строки 
clear_data.head()

#проверим наличие дубликатов в таблице, они могли появиться после 
#обработки столбца с целями кредита

print ('Дубликатов в таблице:', polluted_data.duplicated().sum())
#удалим их и восстановим индексацию
polluted_data = polluted_data.drop_duplicates().reset_index(drop=True) 
#проверим результаты
print ('Дубликатов в таблице после обработки:', polluted_data.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 9 columns):
children            21453 non-null int64
dob_years           21453 non-null int64
education_id        21453 non-null int64
family_status_id    21453 non-null int64
gender              21453 non-null object
income_type         21453 non-null object
debt                21453 non-null int64
total_income        21453 non-null int64
purpose_clear       21453 non-null object
dtypes: int64(6), object(3)
memory usage: 1.5+ MB
None
Дубликатов в таблице: 54
Дубликатов в таблице после обработки: 0


**Вывод**

Сделам лемматизацию мы получили четкий список целей кредита и смогли обнаружить ещё некотрое количество дубликатов

### Категоризация данных

Для ответов нам нужно две категоризации - категоризация по доходу и категоризация по наличию детей. Кроме того я хочу сделать категоризацию по возрасту.
Посмотрим распределения по доходу, найдем разные варианты

In [32]:
#посмотрим характерные значения дохода
clear_data['total_income'].describe()

count    2.145300e+04
mean     1.674319e+05
std      9.806288e+04
min      2.066700e+04
25%      1.076200e+05
50%      1.518760e+05
75%      2.024170e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [33]:
# определим большой доход как 25% процентов самого большого дохода, 
# 25% нижних знначений будут низким доходом, остальное будет средним доходом. 
# Сделаем функцию для разбиения доходов на группы
def income_group(income):
    if income < 106806:
        return 'низкий доход'
    if income < 196968:
        return 'средний доход'
    return 'высокий доход'

clear_data['income_level'] = clear_data['total_income'].apply(income_group) 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Возможно нам потребуется категорирование по возрасту. Я считаю, что разбивать нужно на три группы - до 25 лет (группа не имеет как правило установившегося дохода и большого имущества), до от 25 до 50 лет (группа является основной целевой для кредитов), более 50 лет (группа имеет риски по возрасту и потере работы)

In [41]:
# Сделаем функцию для разбиения возраста на группы
def age_group(age):
        if age < 25:
                return 'молодые люди'
        if age <= 50:
                return 'средний возраст'
        return 'старший возраст'

clear_data['age_level'] = clear_data['dob_years'].apply(age_group)
clear_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
children            21453 non-null int64
dob_years           21453 non-null int64
education_id        21453 non-null int64
family_status_id    21453 non-null int64
gender              21453 non-null object
income_type         21453 non-null object
debt                21453 non-null int64
total_income        21453 non-null int64
purpose_clear       21453 non-null object
income_level        21453 non-null object
age_level           21453 non-null object
having_kids         21453 non-null object
dtypes: int64(6), object(6)
memory usage: 2.0+ MB


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Для ответа на заданные вопросы необходимо однозначно определять наличие детей у заёмщика

In [35]:
# Сделаем функцию для групировки по наличию детей
def have_children(children):
        if children > 0:
                return 'есть дети'
        return 'без детей'

clear_data['having_kids'] = clear_data['children'].apply(have_children)
clear_data.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose_clear,income_level,age_level,having_kids
0,1,42,0,0,F,сотрудник,0,253875,недвижимость,высокий доход,средний возраст,есть дети
1,1,36,1,0,F,сотрудник,0,112080,автомобиль,средний доход,средний возраст,есть дети
2,0,33,1,0,M,сотрудник,0,145885,недвижимость,средний доход,средний возраст,без детей
3,3,32,1,0,M,сотрудник,0,267628,образование,высокий доход,средний возраст,есть дети
4,0,53,1,1,F,пенсионер,0,158616,свадьба,средний доход,старший возраст,без детей


In [36]:
# ещё раз проверим отсутствующие значения
print ('Пропусков в таблице:',clear_data.isna().sum())

Пропусков в таблице: children            0
dob_years           0
education_id        0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose_clear       0
income_level        0
age_level           0
having_kids         0
dtype: int64


**Вывод**

Категоризация данных понадобится нам для ответа на заданные вопросы. 

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Попробуем определить зависит ли невозврат кредита в срок от наличия детей - я предполагаю, что если такая зависимость существует, то она будет очевидна при сравнении доли должников в каждой категории.

In [37]:
#Я хочу сделать общие вычисления, которые могут потом пригодиться.
#Сосчитаем необходимые отношения общие для всех займов и отдельно для невозвращенных займов. 
#Определим обычный процент невозврата  

#проверим значения столбца dedt
print('Значения столбца debt:')
print(clear_data['debt'].value_counts())
#определим количество удачных займов
debt_refund = clear_data.loc[clear_data['debt'] == 0,'purpose_clear'].count()
#определим количество задолженностей
debt_not_refund = clear_data.loc[clear_data['debt'] == 1,'purpose_clear'].count()

#посчитаем проценты для удобства
percentage = (debt_refund + debt_not_refund)/100
#процент удачных и ...
percentage_debt_refund = debt_refund / percentage
#...процент неудачных кредитов
percentage_debt_not_refund = debt_not_refund / percentage
#посчитаем процент среди доли проблемных кредитов
percentage_not_refund = debt_not_refund /100

#выведем результаты на экран
print('')
print('1% в нашей таблице =', percentage)
print('% успешных кредитов =', percentage_debt_refund)
print('% кредитов с просрочкой =',percentage_debt_not_refund)
print('1% среди кредитов с просрочкой =',percentage_not_refund)
print('')

#подсчитаем основную метрику - долю должников в каждой категории
kids_pivot = clear_data.pivot_table(index='having_kids', values='debt')
print(kids_pivot)

Значения столбца debt:
0    19712
1     1741
Name: debt, dtype: int64

1% в нашей таблице = 214.53
% успешных кредитов = 91.88458490653987
% кредитов с просрочкой = 8.115415093460122
1% среди кредитов с просрочкой = 17.41

                 debt
having_kids          
без детей    0.075444
есть дети    0.092082


**Вывод**

Семьи с детьми несколько чаще имеют проблем с возвращением кредита в срок, но эта зависимость не очень ярко выражена (влияние меньше 5%)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Я предполагаю, что если такая зависимость существует, то она будет очевидна при сравнении доли должников в каждой категории.

In [38]:
# Согласно заданию, у нас принят следующий код в столбце family_status_id
#0 – женат/замужем
#1 - гражданский брак
#2 - вдова/вдовец
#3 – в разводе
#4 – не женат/не замужем

#подсчитаем основную метрику - долю должников в каждой категории
family_pivot = clear_data.pivot_table(index='family_status_id', values='debt')
print(family_pivot)

                      debt
family_status_id          
0                 0.075452
1                 0.093494
2                 0.065693
3                 0.071130
4                 0.097509


**Вывод**

Заёмщики без семьи самые опасные с точки зрения невозврата долга. С другой стороны вдовы и вдовцы самая безопасная. Как и те, кто в разводе. При этом мне непонятна разница между людьми в разводе и не женатыми/не замужними. У меня есть совмения, что есть зависимость между семейным положением и возвратом кредита в срок.  

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Я предполагаю, что если такая зависимость существует, то она будет очевидна при сравнении общей доли заёмщиков с долей среди них не вернувших кредит.

In [39]:
#подсчитаем основную метрику - долю должников в каждой категории
income_level_pivot = clear_data.pivot_table(index='income_level', values='debt')
print(income_level_pivot)

                   debt
income_level           
высокий доход  0.070218
низкий доход   0.078803
средний доход  0.088433


**Вывод**

Заёмщики с высоким доходом реже имеют проблемы с возвращением кредита в срок, но эта зависимость не очень ярко выражена (влиянее меньше 5%)

- Как разные цели кредита влияют на его возврат в срок?

In [40]:
# я оставил здесь то, что у пеня было ранее. Это помогло мне разобраться с pivot_table, 
# ниже записал решение через pivot_table

# определим количество проблемных кредитов по каждой цели
auto_not_refund = clear_data[(clear_data['purpose_clear'] == 'автомобиль') & (clear_data['debt'] == 1)]
auto_not_refund_count = auto_not_refund['purpose_clear'].count()

home_not_refund = clear_data[(clear_data['purpose_clear'] == 'недвижимость') & (clear_data['debt'] == 1)]
home_not_refund_count = home_not_refund['income_level'].count()

study_not_refund = clear_data[(clear_data['purpose_clear'] == 'образование') & (clear_data['debt'] == 1)]
study_not_refund_count = study_not_refund['purpose_clear'].count()

mar_not_refund = clear_data[(clear_data['purpose_clear'] == 'свадьба') & (clear_data['debt'] == 1)]
mar_not_refund_count = mar_not_refund['purpose_clear'].count()


# определим относительную проблемность каждой из категорий
auto_percentage_debt = auto_not_refund_count / ((clear_data.loc[clear_data['purpose_clear'] == 'автомобиль','purpose_clear'].count())/100)

home_percentage_debt = home_not_refund_count / ((clear_data.loc[clear_data['purpose_clear'] == 'недвижимость','purpose_clear'].count())/100)

study_percentage_debt = study_not_refund_count / ((clear_data.loc[clear_data['purpose_clear'] == 'образование','purpose_clear'].count())/100)

mar_percentage_debt = mar_not_refund_count / ((clear_data.loc[clear_data['purpose_clear'] == 'свадьба','purpose_clear'].count())/100)


#сведем результат в общую таблицу и выведем на экран
purpose_result = [['% невозврата', auto_percentage_debt, home_percentage_debt, study_percentage_debt, mar_percentage_debt]]
purpose_columns = ['Цель кредита','автомобиль','недвижимость','образование','свадьба']
purpose_result_data = pd.DataFrame(data=purpose_result,columns=purpose_columns)
print(purpose_result_data.head())
print()

#подсчитаем основную метрику - долю должников в каждой категории
purpose_pivot = clear_data.pivot_table(index='purpose_clear', values='debt')
print(purpose_pivot)

   Цель кредита  автомобиль  недвижимость  образование   свадьба
0  % невозврата    9.359034      7.233373     9.220035  8.006888

                   debt
purpose_clear          
автомобиль     0.093590
недвижимость   0.072334
образование    0.092200
свадьба        0.080069


**Вывод**

Самая безопасная из целей получения кредита - недвижимость, скорее всего клиенты более серьёзно подходят к данной цели. Самая опасная цель - автомобиль.

## Шаг 4. Общий вывод

Рассматриваемые зависимости не очень ярко выражены:

- Семьи с детьми несколько чаще имеют проблем с возвращением кредита в срок, но эта зависимость не очень ярко выражена (влиянее меньше 5%)
- Вдовы и вдовцы реже имеют проблемы с возвращением кредита в срок, но эта зависимость не очень ярко выражена (влиянее меньше 5%)
- Заёмщики с высоким доходом реже имеют проблемы с возвращением кредита в срок, но эта зависимость не очень ярко выражена (влиянее меньше 5%)
- Самая безопасная из целей получения кредита - недвижимость, скорее всего клиенты более серьёзно подходят к данной цели. Самая опасная цель - автомобиль.

Соответственно более безопасные заёмщики это вдовы/вдовцы с большим доходом, без детей получающие кредит с целью приобретения недвижимости.
Наименее безопасные это заёмщики с детьми не имеющие семьи, имеющие средний доход и получающие кредит с целью приобретения автомобиля 